In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rc, plot
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, 
GradientBoostingClassifier
from sklearn.model_selection import train_test_split,
KFold, StratifiedKFold, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import precision_recall_curve, 
classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df=pd.read_csv("train.csv")
n=train_df.shape[0]

In [ ]:
train_df.info()
train_df.head()

In [ ]:
test_df=pd.read_csv('test.csv')
ID_test = test_df.PassengerId # запоминаем id 
y = train_df["(target class)"]
df=pd.concat([train_df,test_df]).drop(["(target class)"], axis=1)

# Preprocessing and Analysis


In [ ]:
print('балланс классов:',train_df['(target class)'].value_counts()) # баланс классов
print('кол-во каждого категориального признака:',
      df.dtypes.value_counts()) # Number of each type of column
print('кол-во уникальных экземпляров катег. признаков:',
      df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)) # Number of unique classes in each object column
print('статистики:',df[''].describe()) # хотим вычислить аномалии. аномалию сделать Nan,потом обработать как пропуск
df[''].replace({365243: np.nan}, inplace = True) # Replace the anomalous values with nan

In [ ]:
def missingdata(data):  # пропущенные значения
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    ms=pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    ms= ms[ms["Percent"] > 0]
#     f,ax =plt.subplots(figsize=(8,6))
#     plt.xticks(rotation='90')
#     fig=sns.barplot(ms.index, ms["Percent"],color="green",alpha=0.8)
#     plt.xlabel('Features', fontsize=15)
#     plt.ylabel('Percent of missing values', fontsize=15)
#     plt.title('Percent missing data by feature', fontsize=15)
    return ms
print('пропущенные значения')
missingdata(df)

In [ ]:
df[''].fillna(df[''].mode()[0],
              inplace = True) # заполняем пропуски категориальные
df[''].fillna(df[''].median(), 
              inplace = True) # заполняем пропуски числовые
drop_column = [''] # выбираем признаки которые удалим (лучше не удалять сначала)
df.drop(drop_column, axis=1, inplace = True)

# Визуализация признаков
## Числовые (количественные признаки)

In [ ]:
df.[''].hist(); # гистограмма (плотность распределения)
df.hist(); # гистограмма для всех числовых сразу

In [ ]:
sns.boxplot(df['']); # ящик с усами. точки - выбросы

## Категориальные и бинарные признаки

In [ ]:
df.[''].value_counts()

In [ ]:
sns.countplot(df['']);
# для первых пяти самых популярных
sns.countplot(df[df[''].\
                 isin(df['']value_counts().head().index)]);

##  Взаимодействия
## Числовые - числовые

In [ ]:
# отбираем все признаки в которыхесть слово charge
feat = [f for f in df.columns if 'charge' in f] 
feat
df[feat].hist(); # строим гистограммы для всех  charge

In [ ]:
# матрица для нескольких признаков попарно (для небольшого числа!!!)
sns.pairplot(df[feat]); # вне диагонали - диаграммы рассеяни

In [ ]:
# для двух признаков
plt.scatter(df['признак 1'],df['признак 2'],
           color=df['target'].map({'0': 'green','1': 'red'})); 
plt.xlabel('название оси х $a^2$')
plt.ylabel('название оси y')
plt.legend()

In [ ]:
print('корреляция\n')  # признаки с высокой корреляцией можно удалить
train_df.corrwith(train_df['Survived'],
                  method='spearman').sort_values(ascending=False)

In [ ]:
sns.heatmap(df.corr());

# Числовой - Категориальный (Бинарный)

In [ ]:
sns.boxplot(x='категориальный (и target)', 
            y='числовой признак', data=df); # violinplot

In [ ]:
df.groupby('категориальный(бинарный)')['числовой'].mean()

# Категориальный- Категориальный

In [ ]:
sns.countplot(x='категориальный', hue='targer', data=df);

In [ ]:
# для бустинговых (lgbm, catboost) надо просто выдетлить
#категориальные признаки, для остальных: перекодировать

le = LabelEncoder() # Create a label encoder object (2 варианта признака)
le_count = 0
for col in df:
    if df[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(df[col].unique())) <= 2:
            # Train on the training data
            le.fit(df[col])
            # Transform both training and testing data
            df[col] = le.transform(df[col])
            le_count += 1
print('%d columns were label encoded.' % le_count)

df = pd.get_dummies(df) # one hot encoder (более 2 вариантов признака)

In [ ]:
corr_matrix = train_df.select_dtypes(['int64', 'float64']).corr()
plt.figure(figsize=(20, 20))
ax=sns.heatmap(corr_matrix)

for tick in ax.get_xticklabels():
    tick.set_fontsize(8)
for tick in ax.get_yticklabels():
    tick.set_fontsize(8)

# Feature Engineering

перед созданием bins можно вызвать метод describe (смотреть квантили), чтобы было легче разбивать на группы (концы не включаются!!)

In [ ]:
df['Age_bin'] = pd.cut(df['Age'], bins=[0,14,20,40,120], 
                       labels=['Children','Teenage','Adult','Elder'])
df['Fare_bin'] = pd.cut(df['Fare'], bins=[0,7.91,14.45,31,120],
                        labels=['Low_fare','median_fare', 'Average_fare',
                                'high_fare'])


In [ ]:
dataset = df
dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
import re # для работы с регулярными выражениями, например поиск в строке
# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess',
                                             'Capt', 'Col','Don',
                                             'Dr', 'Major', 'Rev', 
                                             'Sir', 'Jonkheer', 
                                             'Dona'], 'Rare')
dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
df=dataset

In [ ]:
# PCA
from sklearn.preprocessing import scale # анализ главных компонент 
X_scaled = scale(X.iloc[:,:5].dropna(), axis=0) # было 5 стоблцов 
from sklearn.decomposition import PCA # мз 5 мерного перешли в 2 мерное и сохранили 70 процентов дисперсии
pca = PCA(n_components=2, random_state=42) # стало 2 стоблцов
X_pca = pca.fit_transform(X_scaled)
pca.explained_variance_ratio_.sum()

In [ ]:
# Make a new dataframe for polynomial features
poly_features = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                    'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]
poly_target = poly_features['TARGET']
poly_features = poly_features.drop(columns = ['TARGET'])
# Need to impute missing values
poly_features = imputer.fit_transform(poly_features)
from sklearn.preprocessing import PolynomialFeatures
poly_transformer = PolynomialFeatures(degree = 3)
poly_transformer.fit(poly_features)
# Transform the features
poly_features = poly_transformer.transform(poly_features)
print('Polynomial Features shape: ', poly_features.shape)
# Merge polynomial features into training dataframe
poly_features['SK_ID_CURR'] = df['SK_ID_CURR']
df_poly = df.merge(poly_features, on = 'SK_ID_CURR', how = 'left')
# Align the dataframes
df_poly= df_poly.align(app_test_poly, join = 'inner', axis = 1)


# Learning

In [ ]:
test_df=df.iloc[n:,:] #разделить трейн и тест перед обучением
X=df.iloc[:n,:]

In [ ]:
from sklearn.model_selection import GridSearchCV
tree_params = {'max_depth': np.arange(1,11),
              'max_features':[0.5,0.7,1]}

tree_grid=GridSearchCV(first_tree,tree_params,cv=5,n_jobs=4)
tree_grid.fit(X_train,y_train)
tree_grid.best_score_
tree_grid.best_params_
tree_grid.best_estimator_ # лучшая модель 
tree_test_pred=tree.grid.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,tree_test_pred)

In [ ]:
from sklearn.metrics import   #for score!!!!!!!!!!!!!!!

X_train,X_test,y_train,y_test = train_test_split(X,y,
                                                 test_size=0.3,
                                                 random_state=42)
# стратифмцировать!!!
kf = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(train, train_labels)` 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)
average_precision = average_precision_score(y_test, y_pred) # задать скор!!!!!!!!!!!!!!
print('score: {0:0.2f}'.format(
    average_precision))
cv_scores = cross_val_score(rfc, X_train, y_train, cv=kf, scoring='average_precision') # задать скор!!!!!!!!!!!!!!
#print("Cross validation scores:\n\t", "\n\t".join("%.4f" % x for x in cv_scores))
print(" CV score = %.4f" % np.mean(cv_scores))

In [ ]:
cat_col = list(X.select_dtypes(include='object')) #катбусту нужно сообщать о категориальных признаках, это их запоминает
best_model.fit(
   X_train, y_train,
   cat_features=cat_col,
   eval_set=(X_eval, y_eval),
   plot=True
)






from catboost import CatBoostClassifier, cv, Pool
cat=CatBoostClassifier(random_state=42, eval_metric='BalancedAccuracy')
cat.fit(X_train,y_train, verbose=False, plot = True,eval_set=(X_test,y_test))
y_pred1=cat.predict(X_test)
average_precision = average_precision_score(y_test, y_pred1)
print('Average precision-recall score (cat): {0:0.2f}'.format(
      average_precision))
result=cross_val_score(cat,y,Targeted_feature,cv=10,scoring='accuracy')
print('The cross validated score for CatBoostClassifier is:',round(result.mean()*100,2))
y_predict = cross_val_predict(cat,train_df,y,cv=kf)
# задать сv у catboost

In [ ]:
import lightgbm as lgb  # разобраться 
params = {
    'objective' :'binary',
    'learning_rate' : 0.02,
    'num_leaves' : 76,
    'feature_fraction': 0.64, 
    'bagging_fraction': 0.8, 
    'bagging_freq':1,
    'boosting_type' : 'gbdt',
    'metric': 'binary_logloss'
}
 # making lgbm datasets for train and valid
    d_train = lgbm.Dataset(X_train, y_train)
    d_valid = lgbm.Dataset(X_test, y_test)
    
gbm = lgb.train(params, d_train, 5000, valid_sets=[d_valid], verbose_eval=50, early_stopping_rounds=100,feval=binary_error)

# Feature importance

In [ ]:
ans = pd.DataFrame(model.predict(test_df),columns=['(target class)']) 
ans.index = ID_test
ans.to_csv('submission.csv')